## Train MM / explore with random sampling


In [ ]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True

import os
import torch
import numpy as np

from agent import DQNAgent
from tqdm.auto import tqdm
import random
import itertools

# Number of combinations you want
num_combinations = 500  # Change this to however many combinations you need

# default
room_size = "xl-different-prob"
capacity_max = 6
terminates_at = 99
num_iterations = (terminates_at + 1) * 100

prob_type = (
    "non-equal-object-probs" if "different-prob" in room_size else "equal-object-probs"
)
root_path = (
    f"./training-results/{prob_type}/dqn/room_size={room_size}/capacity={capacity_max}/"
)

# root_path = f"training-results/TRASH/{room_size}"

# random
replay_buffer_size_ = [num_iterations // 10]
test_seed_ = [i for i in range(num_combinations)]
target_update_interval_ = [50, 100]
gamma_ = [0.99, 0.9]
semantic_decay_factor_ = [0.8]
pretrain_semantic_ = [False]
relu_between_gcn_layers_ = [False, True]
dropout_between_gcn_layers_ = [False, True]
num_layers_ = [2, 4]
batch_size_ = [32, 64]
embedding_dim_ = [32, 64]
triple_qual_weight_ = [0.8]
intrinsic_explore_reward_ = [0.5, 1.0, 2.0, 5.0, 10]
learning_rate_ = [0.001, 0.0001]
explore_policy_ = ["rl"]
mm_policy_ = ["rl"]
scale_reward_ = [True, False]
gcn_type_ = ["stare"]


# Generate all combinations
params_all = list(
    itertools.product(
        test_seed_,
        target_update_interval_,
        gamma_,
        semantic_decay_factor_,
        pretrain_semantic_,
        replay_buffer_size_,
        relu_between_gcn_layers_,
        dropout_between_gcn_layers_,
        num_layers_,
        batch_size_,
        embedding_dim_,
        triple_qual_weight_,
        intrinsic_explore_reward_,
        learning_rate_,
        explore_policy_,
        mm_policy_,
        scale_reward_,
        gcn_type_,
    )
)

# Random combinations with weighted agent_capacity_
random_combinations = random.sample(params_all, num_combinations)

for i, params in tqdm(enumerate(random_combinations)):
    (
        test_seed,
        target_update_interval,
        gamma,
        semantic_decay_factor,
        pretrain_semantic,
        replay_buffer_size,
        relu_between_gcn_layers,
        dropout_between_gcn_layers,
        num_layers,
        batch_size,
        embedding_dim,
        triple_qual_weight,
        intrinsic_explore_reward,
        learning_rate,
        explore_policy,
        mm_policy,
        scale_reward,
        gcn_type,
    ) = params

    params_dict = {
        "env_str": "room_env:RoomEnv-v2",
        "num_iterations": num_iterations,
        "replay_buffer_size": replay_buffer_size,
        "warm_start": batch_size,
        "batch_size": batch_size,
        "target_update_interval": target_update_interval,
        "epsilon_decay_until": num_iterations,
        "max_epsilon": 1.0,
        "min_epsilon": 0.1,
        "gamma": gamma,
        "learning_rate": learning_rate,
        "capacity": {"long": capacity_max, "short": 15},
        "pretrain_semantic": pretrain_semantic,
        "semantic_decay_factor": semantic_decay_factor,
        "dqn_params": {
            "gcn_layer_params": {
                "type": gcn_type,
                "embedding_dim": embedding_dim,
                "num_layers": num_layers,
                "gcn_drop": 0.1,
                "triple_qual_weight": triple_qual_weight,
            },
            "relu_between_gcn_layers": relu_between_gcn_layers,
            "dropout_between_gcn_layers": dropout_between_gcn_layers,
            "mlp_params": {"num_hidden_layers": num_layers, "dueling_dqn": True},
        },
        "num_samples_for_results": {"val": 5, "test": 10},
        "validation_interval": 1,
        "plotting_interval": 50,
        "train_seed": test_seed + 5,
        "test_seed": test_seed,
        "device": "cpu",
        "qa_function": "latest_strongest",
        "env_config": {
            "question_prob": 1.0,
            "terminates_at": terminates_at,
            "randomize_observations": "all",
            "room_size": room_size,
            "rewards": {"correct": 1, "wrong": 0, "partial": 0},
            "make_everything_static": False,
            "num_total_questions": 1000,
            "question_interval": 1,
            "include_walls_in_observations": True,
        },
        "intrinsic_explore_reward": intrinsic_explore_reward,
        "ddqn": True,
        "default_root_dir": root_path,
        "explore_policy": explore_policy,
        "mm_policy": mm_policy,
        "scale_reward": scale_reward,
    }

    agent = DQNAgent(**params_dict)
    agent.train()

## Run fixed combinations

In [ ]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True

import os
from agent import DQNAgent
from tqdm.auto import tqdm
import random
import itertools


room_size = "xl-different-prob"
terminates_at = 99
num_iterations = (terminates_at + 1) * 200
replay_buffer_size = num_iterations
batch_size = 32
target_update_interval = 10
gamma = {"mm": 0.9, "explore": 0.9}
semantic_decay_factor = 0.8
embedding_dim = 64
num_layers = 2
triple_qual_weight = 0.8

for test_seed in [0, 1, 2, 3, 4]:
    for capacity_max in [96, 192]:
        prob_type = (
            "non-equal-object-probs"
            if "different-prob" in room_size
            else "equal-object-probs"
        )
        root_path = (
            f"./training-results/{prob_type}/dqn/"
            f"room_size={room_size}/capacity={capacity_max}/"
        )
        for pretrain_semantic in [False]:
            params_dict = {
                "env_str": "room_env:RoomEnv-v2",
                "num_iterations": num_iterations,
                "replay_buffer_size": replay_buffer_size,
                "warm_start": batch_size,
                "batch_size": batch_size,
                "target_update_interval": target_update_interval,
                "epsilon_decay_until": num_iterations,
                "max_epsilon": 1.0,
                "min_epsilon": 0.1,
                "gamma": gamma,
                "learning_rate": 0.001,
                "capacity": {"long": capacity_max, "short": 15},
                "pretrain_semantic": pretrain_semantic,
                "semantic_decay_factor": semantic_decay_factor,
                "dqn_params": {
                    "gcn_layer_params": {
                        "type": "stare",
                        "embedding_dim": embedding_dim,
                        "num_layers": num_layers,
                        "gcn_drop": 0.1,
                        "triple_qual_weight": triple_qual_weight,
                    },
                    "relu_between_gcn_layers": True,
                    "dropout_between_gcn_layers": False,
                    "mlp_params": {
                        "num_hidden_layers": num_layers,
                        "dueling_dqn": True,
                    },
                },
                "num_samples_for_results": {"val": 5, "test": 10},
                "validation_interval": 1,
                "plotting_interval": 50,
                "train_seed": test_seed + 5,
                "test_seed": test_seed,
                "device": "cpu",
                "qa_function": "latest_strongest",
                "env_config": {
                    "question_prob": 1.0,
                    "terminates_at": terminates_at,
                    "randomize_observations": "all",
                    "room_size": room_size,
                    "rewards": {"correct": 1, "wrong": 0, "partial": 0},
                    "make_everything_static": False,
                    "num_total_questions": 1000,
                    "question_interval": 1,
                    "include_walls_in_observations": True,
                },
                "intrinsic_explore_reward": 0,
                "ddqn": True,
                "default_root_dir": root_path,
                "explore_policy": "rl",
                "mm_policy": "rl",
                "scale_reward": False,
            }

            agent = DQNAgent(**params_dict)
            agent.train()